In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv, ChebConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
import pdb
from torch_geometric.data import Dataset
from torch_geometric.loader import DataLoader
import torch.optim as optim
from torch_geometric.utils import to_dense_batch

In [2]:
from torch_geometric.data import Data

In [3]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [4]:
class CustomGraphDataset(Dataset):
    def __init__(self, data_list):
        """
        Args:
            data_list (list): 包含多个 CustomGraphData 对象的列表
        """
        super(CustomGraphDataset, self).__init__()
        self.data_list = data_list

    def len(self):
        return len(self.data_list)

    def get(self, idx):
        return self.data_list[idx]

In [5]:
class CustomGraphData(Data):
    def __init__(self, x=None, edge_index=None, edge_attr=None, y=None, class_label=None, min_vals=None, max_vals=None, coord_list=None):
        # 正确调用父类的构造函数
        super(CustomGraphData, self).__init__(x=x, edge_index=edge_index, edge_attr=edge_attr, y=y)
        
        # 添加自定义属性
        self.class_label = class_label
        self.min_vals = min_vals
        self.max_vals = max_vals
        self.coord_list = coord_list

In [6]:
class GCN_Model(nn.Module):
    def __init__(self, input_dimen, hidden_dimen, output_dimen, drop_out=True, K= 4, layer_num = 1):
        super(GCN_Model, self).__init__()
        self.conv1 = ChebConv(in_channels=input_dimen, out_channels=hidden_dimen, K=K)
        self.fc = nn.Linear(hidden_dimen, output_dimen)
        self.acti_func = nn.ReLU()
        #-----------------------------------------------------------------------
        self.drop_out = drop_out
        self.output_dimen = output_dimen

        
        for m in self.children():
            #if isinstance(m, GCNConv):
                #m.lin.weight = init.kaiming_uniform_(m.lin.weight, a=0, nonlinearity='leaky_relu')
                #if m.lin.bias is not None:
                    #m.lin.bias = nn.init.constant_(m.lin.bias, 0)
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0)       
                    
    
    def forward(self, x, edge_index, edge_attr):
        # ----------->TOTAL NODE_NUMBER, 28
        # x: TOTAL NODE_NUMBER, hidden_dimen
        
        gcn_out = self.conv1(x, edge_index, edge_attr)
        if self.drop_out:
            gcn_out = F.dropout(gcn_out, training=self.training, p=0.2)
        gcn_out = self.acti_func(gcn_out)
        gcn_out = self.fc(gcn_out)
        if self.drop_out:
            gcn_out = F.dropout(gcn_out, training=self.training, p=0.2)
        gcn_out = self.acti_func(gcn_out)

        
        return gcn_out

In [7]:
class CNN_1D(nn.Module):
    def __init__(self, output_dimen, hidden_channels_1, hidden_channels_2, drop_out = True):
        super(CNN_1D, self).__init__()
        self.conv_layer = nn.Sequential(
            nn.Conv1d(in_channels = output_dimen, out_channels = hidden_channels_1, kernel_size=7,  stride=1, padding=0),
            nn.LeakyReLU(), #len 22 days
            nn.Conv1d(in_channels = hidden_channels_1, out_channels = hidden_channels_2, kernel_size = 6, stride = 3, padding=1),
            nn.LeakyReLU())  #len: 7 days
        self.acti_func = nn.LeakyReLU()
        self.drop_out = drop_out 

        
        for m in self.modules():
            if isinstance(m, nn.Conv1d) or isinstance(m, nn.ConvTranspose1d) or isinstance(m, nn.Linear):
                # 对卷积层使用 Kaiming 正态初始化
                nn.init.kaiming_uniform_(m.weight.data, nonlinearity='leaky_relu')
                if m.bias is not None:
                    nn.init.constant_(m.bias, 0)
        
       
        
#PGNN处理后数据形状 : TOTAL NODE_NUMBER, hidden_dimen, 28 ------> (Batch_num * Max_node_num), output_dimen, 28
    
    def forward(self, x):
      
        #output shape: TOTAL NODE_NUMBER, hidden_channels_2, 7
        out = self.conv_layer(x) 
        #SHAPE: TOTAL NODE_NUMBER, 7, hidden_channels_2 
        out = out.permute(0,2,1)
 
        #----------------------------------------------------
  
       
        return out

In [8]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_len, drop_out = True):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers = 1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * output_seq_len)
        self.output_seq_len = output_seq_len
        self.output_size = output_size
        self.acti_func = nn.ReLU()
        self.drop_out = drop_out
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0) 
          
    
    def forward(self, x):
        # x shape: (Batch_size * NODE_NUM), 28, output_dimen
        lstm_out, _ = self.lstm(x)
        if self.drop_out:
            lstm_out = F.dropout(lstm_out, training=self.training, p=0.2)
        lstm_out = self.acti_func(lstm_out)
        # lstm_out shape: (Batch_num * Max_node_num), hidden_size 
        lstm_out = lstm_out[:, -1, :]
        out = self.fc(lstm_out)  
        out= self.acti_func(out)
        #out shape: Batch_num, Max_node_num, output_seq_len

        #Batch_num * Max_node_num), output_seq_len
        return out

In [10]:
class RNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, output_seq_len, drop_out = True):
        super(RNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.rnn = nn.RNN(input_size, hidden_size, num_layers=1, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size * output_seq_len)
        self.output_seq_len = output_seq_len
        self.output_size = output_size
        self.acti_func = nn.ReLU()
        self.drop_out = drop_out
        for m in self.modules():
            if isinstance(m, nn.Linear):
                m.weight.data = init.kaiming_uniform_(m.weight.data, nonlinearity='relu')
                if m.bias is not None:
                    m.bias.data = init.constant_(m.bias.data, 0.0) 

    
    def forward(self, x):
        # x shape: (Batch_size * NODE_NUM), 28, output_dimen
        lstm_out, _ = self.rnn(x)
        if self.drop_out:
            lstm_out = F.dropout(lstm_out, training=self.training, p=0.2)
        lstm_out = self.acti_func(lstm_out)
        # lstm_out shape: (Batch_num * Max_node_num), hidden_size 
        lstm_out = lstm_out[:, -1, :]
        out = self.fc(lstm_out)  
        out= self.acti_func(out)
        #out shape: Batch_num, Max_node_num, output_seq_len

        #Batch_num * Max_node_num), output_seq_len
        return out

In [11]:
train_dataset = torch.load("D:/ThesisData/processed data/TargetDomain/NEW/train_GCN_data_14days.h5")

In [12]:
vali_barcelona = torch.load("D:/ThesisData/processed data/TargetDomain/NEW/Barcelona_vali_GCN_data_14days.h5")
test_barcelona = torch.load("D:/ThesisData/processed data/TargetDomain/NEW/Barcelona_test_GCN_data_14days.h5")

In [13]:
batch_size = 14
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
Barcelona_vali_loader = DataLoader(vali_barcelona, batch_size=batch_size, shuffle=False)
Barcelona_test_loader = DataLoader(test_barcelona, batch_size=batch_size, shuffle=False)

In [19]:
#GCN
input_dimen, hidden_dimen, output_dimen = 1, 128, 64
#----------------------------------------------------------------------

#----------------------------------------------------------------------
output_seq_len = 14  # 预测14天
output_size = 1
#LSTM
input_size, hidden_size = output_dimen, 128
#-------------------------------------------------

In [26]:
#lstm = LSTMModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, output_seq_len=output_seq_len)
lstm = RNNModel(input_size=input_size, hidden_size=hidden_size, output_size=output_size, output_seq_len=output_seq_len)
lstm_optimizer = optim.NAdam(lstm.parameters(), lr=3e-4, weight_decay=1e-5)  #4e-4
lstm_scheduler = optim.lr_scheduler.StepLR(lstm_optimizer, step_size=12, gamma=0.6)
#------------------------------------------------------------------------------------------
gcn = GCN_Model(input_dimen, hidden_dimen, output_dimen)
gcn_optimizer = optim.NAdam(gcn.parameters(), lr=4e-4, weight_decay=1e-5)  #5e-4
gcn_scheduler = optim.lr_scheduler.StepLR(gcn_optimizer, step_size=12, gamma=0.55)
#----------------------------------------------------------------------------------------------------
criterion_MSE = nn.MSELoss(reduction='none')  # 使用均方误差作为损失函数
criterion_MAE = nn.L1Loss(reduction='none')

In [27]:
checkpoint = torch.load('D:/ThesisData/processed data/ModelPara/source_Cheb_rnn_14days.pth')
lstm.load_state_dict(checkpoint['lstm_state_dict'])
gcn.load_state_dict(checkpoint['gcn_state_dict'])
#gcn_optimizer.load_state_dict(checkpoint['gcn_optimizer_state_dict'])
#lstm_optimizer.load_state_dict(checkpoint['lstm_optimizer_state_dict'])


<All keys matched successfully>

In [22]:
epoch_num= 40

In [29]:
for h in range(epoch_num):
    lstm.train()
    gcn.train()

    
    for batch in train_loader:

        x, edge_index, edge_attr, y = batch.x, batch.edge_index, batch.edge_attr, batch.y
        batch_min, batch_max = batch.min_vals, batch.max_vals
        #--------------------------------------------------------

        for i in range(x.shape[-1]):
            if i == 0:
                # SHAPE: TOTAL NODE_NUMBER, hidden_dimen, 1
                gcn_out = gcn(x[:, i].unsqueeze(-1), edge_index, edge_attr).unsqueeze(-1)
            else:
                gcn_out = torch.cat((gcn_out, gcn(x[:, i].unsqueeze(-1), edge_index, edge_attr).unsqueeze(-1)), dim=-1)

        # SHAPE: TOTAL NODE_NUMBER, 14
        #----------------------------------------------------------
        gcn_out = gcn_out.permute(0,2,1)
        lstm_out = lstm(gcn_out)
        #----------------------------------------------------------
        

        #-----------------------------------------------------------
        batch_info = batch.batch
        #SHAPE: batch_size, max_num_nodes, 14
        batch_out, mask = to_dense_batch(lstm_out, batch_info)
        #------------------------------------------------------------
        
      
        # batch_labels SHAPE: batch_size, max_num_nodes, 14
        batch_labels, mask = to_dense_batch(y, batch_info)
        # mask SHAPE: batch_size, max_num_nodes
        batch_masks = mask.unsqueeze(-1).repeat(1, 1, 14)
        #-----------------------------------------------------------

        
        loss_mse = criterion_MSE(batch_out, batch_labels) * batch_masks
        loss_mae = criterion_MAE(batch_out, batch_labels) * batch_masks
        loss_mse = loss_mse.sum() / batch_masks.sum()
        loss_mae = loss_mae.sum() / batch_masks.sum()    

        lstm_optimizer.zero_grad()
        gcn_optimizer.zero_grad()

        
        loss_mse.backward()

        
        torch.nn.utils.clip_grad_norm_(lstm.parameters(), 1.0)
        torch.nn.utils.clip_grad_norm_(gcn.parameters(), 1.0)

        lstm_optimizer.step()
        gcn_optimizer.step()

        del loss_mse
        del loss_mae
        #----------------------------------------------------------------------
        
        inverse_outputs = batch_out * (batch_max - batch_min) + batch_min
        inverse_labels = batch_labels * (batch_max - batch_min) + batch_min

        loss_mse = criterion_MSE(inverse_outputs, inverse_labels) * batch_masks
        loss_mae = criterion_MAE(inverse_outputs, inverse_labels) * batch_masks
        loss_mse = loss_mse.sum() / batch_masks.sum()
        loss_mae = loss_mae.sum() / batch_masks.sum() 
    
    
    print(f"epoch {h + 1}, Train Set Inversed Values: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")



#--------------------------------------------------
    lstm.eval()
    gcn.eval()  

   
    with torch.no_grad():
        
        for batch in Barcelona_vali_loader:
            x, edge_index, edge_attr, y = batch.x, batch.edge_index, batch.edge_attr, batch.y
            batch_min, batch_max = batch.min_vals, batch.max_vals
            #--------------------------------------------------------

            for i in range(x.shape[-1]):
                if i == 0:
                    # SHAPE: TOTAL NODE_NUMBER, hidden_dimen, 1
                    gcn_out = gcn(x[:, i].unsqueeze(-1), edge_index, edge_attr).unsqueeze(-1)
                else:
                    gcn_out = torch.cat((gcn_out, gcn(x[:, i].unsqueeze(-1), edge_index, edge_attr).unsqueeze(-1)), dim=-1)
    
            # SHAPE: TOTAL NODE_NUMBER, hidden_dimen, 28
            #----------------------------------------------------------
            gcn_out = gcn_out.permute(0,2,1)
            lstm_out = lstm(gcn_out)
            #----------------------------------------------------------
        
            
            #-----------------------------------------------------------
            batch_info = batch.batch
            #SHAPE: Batch_size, NODE_NUM, 7
            batch_out, mask = to_dense_batch(lstm_out, batch_info)
            #------------------------------------------------------------
        
      
            # batch_labels SHAPE: batch_size, max_num_nodes, 7
            batch_labels, mask = to_dense_batch(y, batch_info)
            # mask SHAPE: batch_size, max_num_nodes
            batch_masks = mask.unsqueeze(-1).repeat(1, 1, 14)     

            
            Barcelona_vali_outputs = batch_out * (batch_max - batch_min) + batch_min
            Barcelona_vali_labels = batch_labels * (batch_max - batch_min) + batch_min

            loss_mse = criterion_MSE(Barcelona_vali_outputs, Barcelona_vali_labels) * batch_masks
            loss_mae = criterion_MAE(Barcelona_vali_outputs, Barcelona_vali_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()  

        print(f"----Vali Barcelona: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")


#--------------------------------------------------
        for batch in Barcelona_test_loader:
            x, edge_index, edge_attr, y = batch.x, batch.edge_index, batch.edge_attr, batch.y
            batch_min, batch_max = batch.min_vals, batch.max_vals
            #--------------------------------------------------------

            for i in range(x.shape[-1]):
                if i == 0:
                    # SHAPE: TOTAL NODE_NUMBER, hidden_dimen, 1
                    gcn_out = gcn(x[:, i].unsqueeze(-1), edge_index, edge_attr).unsqueeze(-1)
                else:
                    gcn_out = torch.cat((gcn_out, gcn(x[:, i].unsqueeze(-1), edge_index, edge_attr).unsqueeze(-1)), dim=-1)
    
            # SHAPE: TOTAL NODE_NUMBER, hidden_dimen, 28
            #----------------------------------------------------------
            gcn_out = gcn_out.permute(0,2,1)
            lstm_out = lstm(gcn_out)
            #----------------------------------------------------------
        
            
            #-----------------------------------------------------------
            batch_info = batch.batch
            #SHAPE: Batch_size, NODE_NUM, 7
            batch_out, mask = to_dense_batch(lstm_out, batch_info)
            #------------------------------------------------------------
        
      
            # batch_labels SHAPE: batch_size, max_num_nodes, 7
            batch_labels, mask = to_dense_batch(y, batch_info)
            # mask SHAPE: batch_size, max_num_nodes
            batch_masks = mask.unsqueeze(-1).repeat(1, 1, 14)     

            
            Barcelona_test_outputs = batch_out * (batch_max - batch_min) + batch_min
            Barcelona_test_labels = batch_labels * (batch_max - batch_min) + batch_min

            loss_mse = criterion_MSE(Barcelona_test_outputs, Barcelona_test_labels) * batch_masks
            loss_mae = criterion_MAE(Barcelona_test_outputs, Barcelona_test_labels) * batch_masks
            loss_mse = loss_mse.sum() / batch_masks.sum()
            loss_mae = loss_mae.sum() / batch_masks.sum()  

        print(f"----Test Barcelona: MSE={loss_mse.item():.1f}, MAE={loss_mae.item():.1f}")
        #------------------------------------------------------------
        


    if h == epoch_num-1:
        cheb_rnn_results = {"Barcelona_vali": Barcelona_vali_outputs.detach().numpy(), "Barcelona_vali_label": Barcelona_vali_labels.detach().numpy(), 
                            "Barcelona_test": Barcelona_test_outputs.detach().numpy(), "Barcelona_test_label": Barcelona_test_labels.detach().numpy(),
                            "Barcelona_node_num": 1273}

        
        save_data(cheb_rnn_results,"D:/ThesisData/processed data/TargetDomain/NEW/results/cheb_lstm_results.h5") 
        break
        
    gcn_optimizer.step()
    lstm_optimizer.step()
    
    
    print("--------------------------------------------")

epoch 1, Train Set Inversed Values: MSE=2580.6, MAE=39.2
----Vali Barcelona: MSE=1245.5, MAE=28.1
----Test Barcelona: MSE=1649.2, MAE=30.5
--------------------------------------------
epoch 2, Train Set Inversed Values: MSE=2622.0, MAE=39.7
----Vali Barcelona: MSE=1259.8, MAE=28.2
----Test Barcelona: MSE=1699.0, MAE=30.8
--------------------------------------------
epoch 3, Train Set Inversed Values: MSE=2550.0, MAE=39.0
----Vali Barcelona: MSE=1268.0, MAE=28.6
----Test Barcelona: MSE=1633.2, MAE=30.6
--------------------------------------------


KeyboardInterrupt: 